In [1]:
import torch
import pandas as pd
import shutil
import io
import numpy as np
import ast
import cv2
import os
from tqdm.auto import tqdm
import shutil as sh
import matplotlib.pyplot as plt

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# path = '/content/drive/MyDrive/hackaton_02.12./trash/'
# new_path = '/content/drive/MyDrive/hackaton_02.12./train_trash/'
# for dir in os.listdir(path):
#   for file in os.listdir(path+dir):
#     #print(path + dir + '_' + file.lower())
#     shutil.copy(path + dir + '/' + file , new_path + dir + '_' + file.lower() )

In [4]:
!git clone https://github.com/ultralytics/yolov5 
!mv yolov5/* ./

Cloning into 'yolov5'...
remote: Enumerating objects: 10237, done.
remote: Total 10237 (delta 0), reused 0 (delta 0), pack-reused 10237
Receiving objects: 100% (10237/10237), 10.40 MiB | 19.79 MiB/s, done.
Resolving deltas: 100% (7107/7107), done.


In [ ]:
!pip install -r requirements.txt

In [5]:
new_path = '/content/drive/MyDrive/hackaton_02.12./train_trash/'
DIR_TRAIN= new_path

dfs = list()

for filename in os.listdir(new_path):
  if '.txt' in filename.lower():
      df = pd.read_csv(new_path + filename, sep=" ", header=None).rename({0:'class_label',1 : 'x_c',2 : 'y_c',3 : 'w',4 : 'h'}, axis=1)
      df['filename'] = filename.lower().replace('.txt', '.png')
      dfs.append(df)

train_df = pd.concat(dfs)

In [6]:
index = list(set(train_df['filename']))
len(index)

600

In [7]:
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//10]
        for name,mini in tqdm(train_df.groupby('filename')):
            if name in val_index:
                path2save = 'val_garb/'
            else:
                path2save = 'train_garb/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+ name.split('.')[0] +".txt", 'w+') as f:
                row = mini[['class_label','x_c','y_c','w','h']].astype(float).values

                # row = row/1024
                
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
                
            sh.copy(f"{DIR_TRAIN}/{name}", 'convertor/fold{}/images/{}/{}'.format(fold,path2save,name))   

100%|██████████| 600/600 [05:25<00:00,  1.84it/s]


In [8]:
%%writefile data.yaml

train: /content/convertor/fold0/images/train_garb/
val: /content/convertor/fold0/images/val_garb/

nc: 2

names: ['Пустой','Переполнен']

Writing data.yaml


In [9]:
!python train.py --img 640 --batch 4 --epochs 10 --data data.yaml --cfg models/yolov5x.yaml --name yolov5x_fold0 --weights yolov5x.pt

train: weights=yolov5x.pt, cfg=models/yolov5x.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolov5x_fold0, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2021-12-2 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7,

In [10]:
!cp -r '/content/runs'  '/content/drive/MyDrive/hackaton_02.12./yolo5_all_content/'
#!cp '/content/runs/train/yolov5x_fold0/weights/best.pt'  '/content/drive/MyDrive/hackaton_02.12./weights/best.pt'

In [11]:
# !cp -r '/content/models'  '/content/drive/MyDrive/hackaton_02.12./yolo5_all_content/'
# !cp -r '/content/convertor'  '/content/drive/MyDrive/hackaton_02.12./yolo5_all_content/'

In [ ]:
!python ./detect.py --weights '/content/runs/train/yolov5x_fold0/weights/best.pt' --img 640 --conf 0.5 --source './convertor/fold0/images/val_garb/'

In [8]:
import PIL
draw_dir = '/content/runs/detect/exp/'

plt.figure(figsize=(50,25))
rows = 4
files = os.listdir(draw_dir)

for num, x in enumerate(files):
    plt.subplot(rows,4,num+1)
    plt.title(x.split('.')[0])
    plt.axis('off')
    plt.imshow(plt.imread(draw_dir+x))